# SERPENS
## An Introduction

In this notebook, I will demonstrate the basic usage of SERPENS.


***
***

### Setup
For a basic simulation setup, we need to specify the following things:
- Define the celestial objects in the _objects.py_ file.
- Define the simulation parameters in the _init.py_ file.

***

#### Setting up parameters

The _init.py_ file contains the _Parameters_ class, which sets the general simulation parameters.
Here we are able to set the default parameters.

Inside the "new" dunder method, we initiate the default _species_ .


In [1]:
from objects import celestial_objects
from species import Species


class Parameters:
    _instance = None

    # Integration specifics
    int_spec = {
        "moon": False,                      # Defines if we deal with a moon. VERY IMPORTANT.
        "sim_advance": 1 / 40,              # Fraction of an orbit. At each fraction multiple, SERPENS adds and modifies particles. We call this an advance.
        "num_sim_advances": 10,             # Total number of advances until a simulation stops.
        "stop_at_steady_state": False,      # CURRENTLY NO EFFECT
        "gen_max": None,                    # Sets the number of advances at which particles are added to the simulation. After this number of advances, the system evolves without adding new particles.
        "r_max": 4,                         # Maximum distance from primary (planet or star) in satellite/planet semi-major axes. Particles further away get removed.
        "random_walk": False,               # CURRENTLY NO EFFECT
        "particle_interpolation": False     # CURRENTLY NO EFFECT
    }

    # Thermal evaporation parameters
    therm_spec = {
        "source_temp_max": 2703,  # 125, #2703, 130
        "source_temp_min": 1609,  # 50, #1609, 90
        "spherical_symm_ejection": True,
    }

    celest = celestial_objects(int_spec["moon"])
    species = {}
    num_species = 0

    def __new__(cls):

        if cls._instance is None:

            cls.species[f"species1"] = Species("H2", description="H2--6.69kg/s--1200m/s", n_th=0, n_sp=1000, mass_per_sec=6.69, model_smyth_v_b=1200, model_smyth_v_M=10000)
            cls.species[f"species2"] = Species("H", description="H--5.845kg/s--2500m/s", n_th=0, n_sp=500, mass_per_sec=5.845, model_smyth_v_b=2500, model_smyth_v_M=10000)
            cls.species[f"species3"] = Species("O2", description="O2--14.35kg/s--4700m/s", n_th=0, n_sp=500, mass_per_sec=14.35, model_smyth_v_b=4700, model_smyth_v_M=10000)
            cls.num_species = len(cls.species)

            cls._instance = object.__new__(cls)

        return cls._instance


#### Setting up celestial objects

The _objects.py_ file contains the _celestial_objects_ function, contain a dictionary of celestial objects.

We distinguish between systems containing a moon-source and system with a planet-source. If you want to change the celestial objects in a system, in which you want to simulate a moon,
make sure to modify the entries under the _if moon_ clause.
SERPENS allows to set default parameters for both cases.

Very important is to define which object is the source. In case of an exomoon system, you need to add _"hash": "moon"_ to the source's characteristic dictionary.
In all cases you need to add the _"hash": "planet"_ dictionary entry to the outgassing planet / exomoon primary.
If you analyze an exomoon system, but don't have an object with entry _"hash": "moon"_, SERPENS throws an error. It also throws an error if there's no object with _"hash": "planet"_.

ATTENTION: Even if you add additional moons/planets, only add the "moon"/"planet" hashes to ONE of those objects. SERPENS identifies the source and primary with these exact hashes.
Additional objects may carry any hash except "moon"/"planet".

You can also define multiple celestial object system, which is import if you want to schedule multiple SERPENS simulations in different exoplanet systems.
Make sure to define the dictionaries with _celest{X}_, where _{X}_ is a set number. This set number can later be put as an argument (see function arguments).

Example:

In [2]:
def celestial_objects(moon, set=1):
    m_sol = 1.988e30
    m_jup = 1.898e27
    r_sol = 696340e3
    r_jup = 69911e3
    au = 149597870700

    if moon:
        # Jupiter
        # -------
        celest1 = {
            "star": {"m": 1.988e30,
                     "hash": 'star',
                     "r": 696340000
                     },
            "planet": {"m": 1.898e27,
                       "a": 7.785e11,
                       "e": 0.0489,
                       "inc": 0.0227,
                       "r": 69911000,
                       "primary": 'star',
                       "hash": 'planet'
                       },
            "Europa": {"m": 4.799e22,  # Europa
                     "a": 6.709e8,
                     "e": 0.009,
                     "inc": 0.0082,
                     "r": 1560800,
                     "primary": 'planet',
                     "hash": 'moon'
                     },
            "Io": {"m": 8.932e22,  # Io
                    "a": 4.217e8,
                    "e": 0.0041,
                    "r": 1821600,
                    "primary": 'planet'
                     }
        }
    else:
        # 55 Cnc-e
        # --------
        celest1 = {
            "star": {"m": 1.799e30,
                     "hash": 'star',
                     "r": 6.56e8
                     },
            "planet": {"m": 4.77179e25,
                       "a": 2.244e9,
                       "e": 0.05,
                       "inc": 0.00288,
                       "r": 1.196e7,
                       "primary": 'star',
                       "hash": 'planet'}
        }

In the above code we set two default systems.
If we set _int_spec["moon"] = True_ in the _Parameters_ class, then the Jovian system will be set as default. Note how only Europa has the hash "moon".
If we set _int_spec["moon"] = False_, then the 55 Cnc-e system will be set as default.


#### Running

Once the objects and parameters have been set up, one can run the _serpens_simulation.py_ directly or simply import the _SerpensSimulation_ class from it.
It then can be initialized and advanced.

In [3]:
from serpens_simulation import SerpensSimulation
from init import Parameters

num_advances = Parameters.int_spec["num_sim_advances"]
ssim = SerpensSimulation()
ssim.advance(num_advances)        # Activate this line to run the simulation!

SERPENS simulation has been created.
Initializing new simulation instance...
	 	 ... done!
Starting advance 0 ... 
	 MP Processes joined.
	 Transfering particle data...
Advance done! 
Simulation time: 0.42
Simulation runtime: 0.41
Number of particles removed: 59
Number of particles: 819
Saving hash dict...
	 ... done!
#######################################################
Starting advance 1 ... 
	 MP Processes joined.
	 Transfering particle data...
Advance done! 
Simulation time: 0.85
Simulation runtime: 0.87
Number of particles removed: 71
Number of particles: 1624
Saving hash dict...
	 ... done!
#######################################################
Starting advance 2 ... 
	 MP Processes joined.
	 Transfering particle data...
Advance done! 
Simulation time: 1.27
Simulation runtime: 1.41
Number of particles removed: 88
Number of particles: 2406
Saving hash dict...
	 ... done!
#######################################################
Starting advance 3 ... 
	 MP Processes joined.
	 Tra

***

### Species

This section covers a quick explanation to the species class.

Species(
> name: str,
>> The name of the species. See the _species.py_ file for all implemented species names, e.g. "Na", "H2", "SO2".

> description: str,
>> If given, this description will be forwarded to any plots and allows a more sophisticated labeling, eg. "H2--6.69kg/s--1200m/s".

> n_th: int,
>> Number of thermal evaporated superparticles added to the simulation per advance.

> n_sp: int,
>> Number of sputtered superparticles added to the simulation per advance.

> mass_per_sec: float,
>> Mass flow rate in kg/s. Needed for density estimations.

> duplicate: int,
>> If multiple species of the same kind are analyzed, but with different parameters, this parameter must be increased by one for each duplicate species starting at 1.

> beta: float,
>> Radiation force to gravitational force ratio parameter.

> lifetime: float,
>> Lifetime of the species.

> n_e: float,
>> Electron density scaling factor. Will be passed to the chemical network, where all reaction rates given per electron density, will be scaled accordingly.

> sput_spec: dict
>> Contains all the parameters for the sputtering distribution. Most important are _model_smyth_v_b_ and _model_smyth_v_M_ which define most likely and maximum speeds, respectively.
)



#### Adding new species

If you want to add new species, there are a few steps you need to undertake:
1. Add species name to _implemented_species_ in the _Species_ class from the _species.py_ file and assign it a number acting as an id.
2. In _species.py_, there's a code block starting with _if name in self.implementedSpecies:_. Inside this block you need to make a new entry similar to what you can see already exists. The number passed to _super()_ is the mass number of the species.
3. Define a network or lifetime in the _network.py_ file that is associated to the new species id.

***

### The scheduler

In case you want to run multiple simulations, you can define new parameters for each simulation in the _scheduler.py_ file.
You need to set a dictionary _simulations_ with the new parameters, which overwrite the default ones saved in the _Parameters_ class.

Possible changes:

species : list
> New species to analyze.
>>  Signature:
>>  _Species(name: str, n_th: int, n_sp: int, mass_per_sec: float, duplicate: int, beta: float, lifetime: float, n_e: float, sput_spec: dict)_
>>  If attribute not passed to species, it is either set to 'None' or to default value.

objects: dict
> Manipulate celestial objects.
>   If passed 'objects: dict', a new source can be defined and object properties can be changed (see rebound.Particle properties)
>>  Example: {'Io': {'source': True, 'm': 1e23, 'a': 3e9}, 'Ganymede': {...}}

moon: bool
>    Change between moon and planet systems (e.g. Jovian to 55 Cnc)
>    NOTE: It is currently only possible to change the chem. network via lifetime between systems.

int_spec: dict
>    update integration parameters. You can also pass only the specific parameter (still in form of a dict).

therm_spec: dict
>    update thermal evaporation parameters. You can also pass only the specific parameter (still in form of a dict).

celest_set: int
>    Change celestial object set according to objects.py.
>    NOTE: Only works if "moon: bool" is also defined. Needed to distinguish between sets


The following example changes to the system with celestial objects _celest3_ (set 3), which represents HD-189733. In order to switch to this exomoon system, we need to state that _moon = True_
We also analyze sodium (instead whatever the default is) with the corresponding species parameters. If we wanted to, we could add more list entries with additional species.
If you run the _scheduler.py_ with this set up, the simulation starts automatically. If _simulations_ had another dictionary entry, it would follow with that simulation and save it in another folder.

In [4]:
from init import NewParams
from species import Species

simulations = {
    "HD-189733-ExoEarth":
        NewParams(species=[Species('Na', description='Na--11576kg/s--2.5-30km/s', n_th=0, n_sp=1000, mass_per_sec=11576,
                                   lifetime=16.9 * 60, model_smyth_v_b=2500, model_smyth_v_M=30000)],
                  moon=True,
                  celest_set=3)
}

***
***

### Analyzing the results

In order to visualize the results, one calls the _SerpensAnalyzer_.
Note that you might need to check your matplotlib interactive framework.

To analyze, you need to have run the simulation, s.t. parameters, hash dictionary and simulation archives have been saved to the working directory.

ATTENTION: I am currently translating _serpens.py_ into _SerpensAnalyzer_.
Currently, there's only the top-down/birdseye plot available.


In [1]:
from serpens_analyzer import SerpensAnalyzer

sa = SerpensAnalyzer()
sa.birdseye(timestep=3, d=2, grid=True, colormesh=True, triplot=False)      # timestep: simulation advance number; triplot: plot tessellation.

ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running